# Spectrum

In [1]:
import pandas as pd
import astropy.units as u
import numpy as np
import re
import matplotlib.pyplot as plt
from astroquery.simbad import Simbad
from astropy.coordinates import SkyCoord
from pandas.plotting import scatter_matrix
from definitions import *

In [2]:
catalog = pd.read_csv("markeddata/gevtevsimbad_TeVspectrum.txt", sep='\t', index_col=0)
catalog.shape

(68, 199)

In [3]:
xmm_catalog = pd.read_csv("data/gevtevsimbadxmm_005.txt", sep='\t', index_col=0)
xmm_catalog.shape

(1343, 214)

In [4]:
xmm_catalog[list_xmm_spectra_columns()]

,xmm_SC_EP_1_FLUX,xmm_SC_EP_2_FLUX,xmm_SC_EP_3_FLUX,xmm_SC_EP_4_FLUX,xmm_SC_EP_5_FLUX
0,2.174950e-15,1.739610e-14,1.299650e-13,8.058750e-14,5.216310e-16
1,1.870300e-16,2.424990e-14,1.522230e-13,9.815520e-14,0.000000e+00
2,6.634830e-16,3.346220e-14,1.715350e-14,8.189470e-16,0.000000e+00
3,1.335640e-15,4.964600e-14,8.610680e-14,3.178850e-14,7.576820e-17
4,1.330540e-14,2.412180e-13,6.972420e-13,4.342810e-13,2.017640e-15
5,2.011410e-15,4.419800e-14,9.614620e-14,1.997430e-14,6.865940e-15
6,5.958310e-16,2.084590e-14,8.782340e-14,2.969430e-14,8.491040e-16
7,3.048430e-14,4.985300e-13,9.787230e-13,4.369890e-13,1.240990e-15
8,1.388830e-15,3.766660e-14,8.333950e-14,5.357300e-14,0.000000e+00
9,2.208710e-15,3.867220e-14,6.624080e-14,3.287630e-14,0.000000e+00


In [5]:
# columns contain spectrums in xmm
xmm_spectrum_columns = list_xmm_spectra_columns()

In [6]:
catalog = catalog.join(xmm_catalog.loc[catalog.index, xmm_spectrum_columns])
catalog

,level_0,gev_0FGL_Name,gev_1FGL_Name,gev_1FHL_Name,gev_2FGL_Name,gev_ASSOC1,gev_ASSOC2,gev_ASSOC_GAM1,gev_ASSOC_GAM2,gev_ASSOC_GAM3,...,tev_0.3TeV,tev_1TeV,tev_3TeV,tev_10TeV,tev_30TeV,xmm_SC_EP_1_FLUX,xmm_SC_EP_2_FLUX,xmm_SC_EP_3_FLUX,xmm_SC_EP_4_FLUX,xmm_SC_EP_5_FLUX
0,83,b' ',b' ',b' ',b' ',b' ',b' ',b' ',b' ',b' ',...,3.341668e-11,2.440195e-12,1.828992e-13,5.610705e-15,6.128219e-17,2.174950e-15,1.739610e-14,1.299650e-13,8.058750e-14,5.216310e-16
1,92,b' ',b' ',b' ',b' ',b'HESS J1841-055 ',b' ',b' ',b' ',b' ',...,6.310590e-11,1.207092e-11,8.365091e-13,4.356098e-14,2.865129e-15,1.870300e-16,2.424990e-14,1.522230e-13,9.815520e-14,0.000000e+00
2,83,b' ',b' ',b' ',b' ',b' ',b' ',b' ',b' ',b' ',...,3.341668e-11,2.440195e-12,1.828992e-13,5.610705e-15,6.128219e-17,6.634830e-16,3.346220e-14,1.715350e-14,8.189470e-16,0.000000e+00
3,86,b' ',b'1FGL J1746.4-2849c',b'1FHL J1746.3-2851 ',b'2FGL J1746.6-2851c',b'PWN G0.13-0.11 ',b' ',b' ',b'3EG J1746-2851',b' ',...,1.647138e-11,8.320851e-14,2.778103e-14,1.856180e-15,3.184437e-16,1.335640e-15,4.964600e-14,8.610680e-14,3.178850e-14,7.576820e-17
4,83,b' ',b' ',b' ',b' ',b' ',b' ',b' ',b' ',b' ',...,3.341668e-11,2.440195e-12,1.828992e-13,5.610705e-15,6.128219e-17,1.330540e-14,2.412180e-13,6.972420e-13,4.342810e-13,2.017640e-15
5,85,b' ',b'1FGL J1746.4-2849c',b'1FHL J1746.3-2851 ',b'2FGL J1746.6-2851c',b'PWN G0.13-0.11 ',b' ',b' ',b'3EG J1746-2851',b' ',...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,2.011410e-15,4.419800e-14,9.614620e-14,1.997430e-14,6.865940e-15
6,86,b' ',b'1FGL J1746.4-2849c',b'1FHL J1746.3-2851 ',b'2FGL J1746.6-2851c',b'PWN G0.13-0.11 ',b' ',b' ',b'3EG J1746-2851',b' ',...,1.647138e-11,8.320851e-14,2.778103e-14,1.856180e-15,3.184437e-16,5.958310e-16,2.084590e-14,8.782340e-14,2.969430e-14,8.491040e-16
7,103,b'0FGL J2032.2+4122',b'1FGL J2032.2+4127 ',b'1FHL J2032.1+4125 ',b'2FGL J2032.2+4126 ',b'LAT PSR J2032+4127 ',b' ',b'1AGL J2032+4102',b' ',b' ',...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,3.048430e-14,4.985300e-13,9.787230e-13,4.369890e-13,1.240990e-15
8,103,b'0FGL J2032.2+4122',b'1FGL J2032.2+4127 ',b'1FHL J2032.1+4125 ',b'2FGL J2032.2+4126 ',b'LAT PSR J2032+4127 ',b' ',b'1AGL J2032+4102',b' ',b' ',...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.388830e-15,3.766660e-14,8.333950e-14,5.357300e-14,0.000000e+00
9,11,b'0FGL J0240.3+6113',b'1FGL J0240.5+6113 ',b'1FHL J0240.2+6113 ',b'2FGL J0240.5+6113 ',b'LS I+61 303 ',b' ',b'1AGL J0242+6111',b' ',b'EGR J0240+6112 ',...,3.033065e-11,5.807977e-13,5.926255e-14,1.818965e-15,2.823374e-16,2.208710e-15,3.867220e-14,6.624080e-14,3.287630e-14,0.000000e+00


Columns contained spectrum in gev:

In [7]:
gev_spectrum_columns = list_gev_spectrum_columns()

Columns contained spectrum in simbad:

In [8]:
s_spectrum_columns = list_s_spectrum_columns()

Columns contained spectrum in tev:

In [9]:
tev_spectrum_columns = list_tev_spectrum_columns()

In [10]:
s_other_columns = list_s_other_columns()

In [11]:
gev_other_columns = list_gev_other_columns()

In [12]:
tev_other_columns = list_tev_other_columns()

In [13]:
xmm_other_columns = list_xmm_other_columns()

In [14]:
spectrum_columns = gev_spectrum_columns + s_spectrum_columns + tev_spectrum_columns
spectrum_with_xmm_columns = spectrum_columns + xmm_spectrum_columns
other_columns = s_other_columns + gev_other_columns + tev_other_columns + ['marked']

In [15]:
spectrums = catalog[spectrum_columns + other_columns].copy()

The next cell computes simbad spectrum using this formula:
$$ F_{simbad} = 10^{-flux_{simbad}}$$

In [16]:
#spectrums[s_spectrum_columns] = 10 ** (-spectrums[s_spectrum_columns])

The next cell computes TeV spectrum using:
$$F_{TeV} = 1.6*E^{2}*Flux$$

In [16]:
#for i in tev_spectrum_columns:
#    if len(re.findall("\d+", i)) == 2:
#        E = 0.3
#    else:
#        E = int(re.findall("\d+", i)[0])
#    spectrums[i] = 1.6 * E ** 2 * spectrums[i]

In [17]:
spectrums

,gev_nuFnu10000_100000,gev_nuFnu1000_3000,gev_nuFnu100_300,gev_nuFnu3000_10000,gev_nuFnu300_1000,gev_nuFnu30_100,s_FLUX_U,s_FLUX_B,s_FLUX_V,s_FLUX_R,...,s_nan_spectra,gev_1FGL_Name,gev_CLASS1,gev_GLON,gev_GLAT,tev_fermi_names,tev_classes,tev_glat,tev_glon,marked
0,2.938044e-12,7.446349e-11,4.520060e-12,3.103834e-11,7.171538e-11,NaN,NaN,NaN,NaN,NaN,...,False,b' ',b' ',359.863953,-0.020071,b'',b'unid',-0.046162,359.944244,0
1,2.098420e-11,2.096862e-11,1.266728e-12,2.274123e-11,7.566604e-12,NaN,NaN,11.160,10.510000,NaN,...,False,b' ',b'PWN ',26.795933,-0.198590,b'',b'unid',-0.197604,26.795113,0
2,2.938044e-12,7.446349e-11,4.520060e-12,3.103834e-11,7.171538e-11,NaN,NaN,NaN,NaN,NaN,...,False,b' ',b' ',359.863953,-0.020071,b'',b'unid',-0.046162,359.944244,0
3,4.706814e-12,5.332763e-11,5.145968e-13,2.923599e-11,3.387356e-11,NaN,NaN,NaN,NaN,NaN,...,False,b'1FGL J1746.4-2849c',b'pwn ',0.149201,-0.103157,b'',b'unid',-0.113206,0.140617,0
4,2.938044e-12,7.446349e-11,4.520060e-12,3.103834e-11,7.171538e-11,NaN,NaN,NaN,NaN,NaN,...,False,b' ',b' ',359.863953,-0.020071,b'',b'unid',-0.046162,359.944244,0
5,4.706814e-12,5.332763e-11,5.145968e-13,2.923599e-11,3.387356e-11,NaN,NaN,NaN,NaN,NaN,...,False,b'1FGL J1746.4-2849c',b'pwn ',0.149201,-0.103157,b'',b'unid',-0.147977,0.054990,0
6,4.706814e-12,5.332763e-11,5.145968e-13,2.923599e-11,3.387356e-11,NaN,NaN,NaN,NaN,NaN,...,False,b'1FGL J1746.4-2849c',b'pwn ',0.149201,-0.103157,b'',b'unid',-0.113206,0.140617,0
7,2.845686e-12,4.600289e-11,1.418847e-11,3.224835e-11,3.170300e-11,NaN,13.120000,13.080,11.950000,NaN,...,False,b'1FGL J2032.2+4127 ',b'PSR ',80.218597,1.018827,b'',b'unid',1.111117,80.299477,1
8,2.845686e-12,4.600289e-11,1.418847e-11,3.224835e-11,3.170300e-11,NaN,12.390000,12.010,11.300000,11.05,...,False,b'1FGL J2032.2+4127 ',b'PSR ',80.218597,1.018827,b'',b'unid',1.111117,80.299477,0
9,5.929003e-12,8.840736e-11,1.353836e-10,4.055951e-11,1.317918e-10,NaN,11.270000,11.610,10.750000,10.19,...,False,b'1FGL J0240.5+6113 ',b'HMB ',135.678848,1.086277,b'',b'bin',1.086135,135.675278,1


In [18]:
spectrums = spectrums.fillna(0)
spectrums = np.log(spectrums[spectrum_columns] + 1)
# spectrums = np.log(spectrums[spectrum_columns])
# spectrums[(spectrums == -np.inf)] = 0

In [19]:
(spectrums == -np.inf).any()

gev_nuFnu10000_100000    False
gev_nuFnu1000_3000       False
gev_nuFnu100_300         False
gev_nuFnu3000_10000      False
gev_nuFnu300_1000        False
gev_nuFnu30_100          False
s_FLUX_U                 False
s_FLUX_B                 False
s_FLUX_V                 False
s_FLUX_R                 False
s_FLUX_I                 False
s_FLUX_G                 False
s_FLUX_J                 False
s_FLUX_H                 False
s_FLUX_K                 False
s_FLUX_u                 False
s_FLUX_g                 False
s_FLUX_r                 False
s_FLUX_i                 False
s_FLUX_z                 False
tev_0.3TeV               False
tev_1TeV                 False
tev_3TeV                 False
tev_10TeV                False
tev_30TeV                False
dtype: bool

In [20]:
display(spectrums)

,gev_nuFnu10000_100000,gev_nuFnu1000_3000,gev_nuFnu100_300,gev_nuFnu3000_10000,gev_nuFnu300_1000,gev_nuFnu30_100,s_FLUX_U,s_FLUX_B,s_FLUX_V,s_FLUX_R,...,s_FLUX_u,s_FLUX_g,s_FLUX_r,s_FLUX_i,s_FLUX_z,tev_0.3TeV,tev_1TeV,tev_3TeV,tev_10TeV,tev_30TeV
0,2.938094e-12,7.446355e-11,4.520162e-12,3.103828e-11,7.171530e-11,0.0,0.000000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.000000,0.000000,0.0,3.341660e-11,2.440270e-12,1.829648e-13,5.551115e-15,0.000000e+00
1,2.098410e-11,2.096856e-11,1.266764e-12,2.274114e-11,7.566614e-12,0.0,0.000000,2.498152,2.443216,0.000000,...,0.0,0.000000,0.000000,0.000000,0.0,6.310596e-11,1.207101e-11,8.364420e-13,4.352074e-14,2.886580e-15
2,2.938094e-12,7.446355e-11,4.520162e-12,3.103828e-11,7.171530e-11,0.0,0.000000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.000000,0.000000,0.0,3.341660e-11,2.440270e-12,1.829648e-13,5.551115e-15,0.000000e+00
3,4.706902e-12,5.332756e-11,5.146994e-13,2.923595e-11,3.387357e-11,0.0,0.000000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.000000,0.000000,0.0,1.647149e-11,8.326673e-14,2.775558e-14,1.776357e-15,2.220446e-16
4,2.938094e-12,7.446355e-11,4.520162e-12,3.103828e-11,7.171530e-11,0.0,0.000000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.000000,0.000000,0.0,3.341660e-11,2.440270e-12,1.829648e-13,5.551115e-15,0.000000e+00
5,4.706902e-12,5.332756e-11,5.146994e-13,2.923595e-11,3.387357e-11,0.0,0.000000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.000000,0.000000,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
6,4.706902e-12,5.332756e-11,5.146994e-13,2.923595e-11,3.387357e-11,0.0,0.000000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.000000,0.000000,0.0,1.647149e-11,8.326673e-14,2.775558e-14,1.776357e-15,2.220446e-16
7,2.845724e-12,4.600298e-11,1.418843e-11,3.224843e-11,3.170308e-11,0.0,2.647592,2.644755,2.561096,0.000000,...,0.0,0.000000,0.000000,0.000000,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
8,2.845724e-12,4.600298e-11,1.418843e-11,3.224843e-11,3.170308e-11,0.0,2.594508,2.565718,2.509599,2.489065,...,0.0,0.000000,0.000000,0.000000,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
9,5.929035e-12,8.840728e-11,1.353837e-10,4.055956e-11,1.317917e-10,0.0,2.507157,2.534490,2.463853,2.415020,...,0.0,0.000000,0.000000,0.000000,0.0,3.033063e-11,5.808687e-13,5.928591e-14,1.776357e-15,2.220446e-16


Normalization:

In [21]:
spectrums[spectrum_columns] = spectrums[spectrum_columns].div(spectrums[spectrum_columns].sum(axis=1), axis=0)

The cell displays all information we have about spectrums now.

In [22]:
display(spectrums)

,gev_nuFnu10000_100000,gev_nuFnu1000_3000,gev_nuFnu100_300,gev_nuFnu3000_10000,gev_nuFnu300_1000,gev_nuFnu30_100,s_FLUX_U,s_FLUX_B,s_FLUX_V,s_FLUX_R,...,s_FLUX_u,s_FLUX_g,s_FLUX_r,s_FLUX_i,s_FLUX_z,tev_0.3TeV,tev_1TeV,tev_3TeV,tev_10TeV,tev_30TeV
0,3.713254e-13,9.410932e-12,5.712720e-13,3.922714e-12,9.063600e-12,0.0,0.000000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.000000,0.000000,0.0,4.223293e-12,3.084089e-13,2.312365e-14,7.015670e-16,0.000000e+00
1,1.457995e-12,1.456915e-12,8.801598e-14,1.580076e-12,5.257354e-13,0.0,0.000000,0.173574,0.169757,0.000000,...,0.0,0.000000,0.000000,0.000000,0.0,4.384662e-12,8.387052e-13,5.811678e-14,3.023862e-15,2.005623e-16
2,3.857000e-13,9.775245e-12,5.933868e-13,4.074568e-12,9.414467e-12,0.0,0.000000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.000000,0.000000,0.0,4.386784e-12,3.203479e-13,2.401880e-14,7.287258e-16,0.000000e+00
3,6.195480e-13,7.019264e-12,6.774753e-14,3.848194e-12,4.458623e-12,0.0,0.000000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.000000,0.000000,0.0,2.168067e-12,1.096002e-14,3.653340e-15,2.338137e-16,2.922672e-17
4,5.668755e-13,1.436699e-11,8.721194e-13,5.988522e-12,1.383674e-11,0.0,0.000000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.000000,0.000000,0.0,6.447395e-12,4.708254e-13,3.530119e-14,1.071031e-15,0.000000e+00
5,6.329856e-13,7.171508e-12,6.921694e-14,3.931660e-12,4.555329e-12,0.0,0.000000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.000000,0.000000,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
6,6.329856e-13,7.171508e-12,6.921694e-14,3.931660e-12,4.555329e-12,0.0,0.000000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.000000,0.000000,0.0,2.215091e-12,1.119774e-14,3.732579e-15,2.388850e-16,2.986063e-17
7,1.639420e-13,2.650229e-12,8.173946e-13,1.857830e-12,1.826413e-12,0.0,0.152528,0.152364,0.147545,0.000000,...,0.0,0.000000,0.000000,0.000000,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
8,1.456778e-13,2.354976e-12,7.263315e-13,1.650856e-12,1.622939e-12,0.0,0.132818,0.131344,0.128471,0.127420,...,0.0,0.000000,0.000000,0.000000,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
9,2.772673e-13,4.134307e-12,6.331127e-12,1.896740e-12,6.163149e-12,0.0,0.117246,0.118524,0.115220,0.112937,...,0.0,0.000000,0.000000,0.000000,0.0,1.418391e-12,2.716393e-14,2.772465e-15,8.307013e-17,1.038377e-17


In [23]:
# spectrums.to_csv("markeddata/normalized_spectra.txt", index=False)

In [23]:
def normalize_spectra(catalog_name, s_spectrum_columns, gev_spectrum_columns, tev_spectrum_columns,
                      s_other_columns, gev_other_columns, tev_other_columns, log = True):
    other_columns = s_other_columns + gev_other_columns + tev_other_columns + ['marked']
    catalog = pd.read_csv("markeddata/" + catalog_name + ".txt", sep='\t', index_col=0)
    
    spectrum_columns =  s_spectrum_columns +gev_spectrum_columns + tev_spectrum_columns
    spectrums = catalog[spectrum_columns + other_columns].copy()
    spectrums = spectrums.fillna(0)
    if log:
        only_spectra = spectrums[spectrum_columns].copy()
        only_spectra = np.log(only_spectra + 1)
#         only_spectra = np.log(only_spectra)
#         only_spectra[(only_spectra == -np.inf)] = 0
        spectrums[spectrum_columns] = only_spectra
    spectrums[spectrum_columns] = spectrums[spectrum_columns].div(spectrums[spectrum_columns].sum(axis=1), axis=0)
    spectrums = spectrums.fillna(0)
    
    display(spectrums)
    if ~log:
        add_name = "WITHOUT_LOG"
    spectrums.to_csv("markeddata/" + catalog_name + "_normalized" + add_name + ".txt", index=False, sep='\t')

In [24]:
def calculate_spectra_old(catalog_name, s_spectrum_columns, gev_spectrum_columns, tev_spectrum_columns,
                      s_other_columns, gev_other_columns, tev_other_columns):
    other_columns = s_other_columns + gev_other_columns + tev_other_columns + ['marked']
    catalog = pd.read_csv("markeddata/" + catalog_name + ".txt", sep='\t', index_col=0)
    
    spectrum_columns =  s_spectrum_columns +gev_spectrum_columns + tev_spectrum_columns
    spectrums = catalog[spectrum_columns + other_columns].copy()
    spectrums = spectrums.fillna(0)
    if log:
        only_spectra = spectrums[spectrum_columns].copy()
        only_spectra = np.log(only_spectra)
        only_spectra[(only_spectra == -np.inf)] = 0
        spectrums[spectrum_columns] = only_spectra
    spectrums[spectrum_columns] = spectrums[spectrum_columns].div(spectrums[spectrum_columns].sum(axis=1), axis=0)
    spectrums = spectrums.fillna(0)
    display(spectrums)
    if ~log:
        add_name = "WITHOUT_LOG"
    spectrums.to_csv("markeddata/" + output_catalog_name + "_normalized" + add_name + ".txt", index=False, sep='\t')

In [25]:
def calculate_spectra(catalog_name, xmm_catalog_name, output_catalog_name, 
                      s_spectrum_columns, gev_spectrum_columns, tev_spectrum_columns, xmm_spectrum_columns,
                      spectrum_columns, s_other_columns, gev_other_columns, tev_other_columns, log=True):
    other_columns = s_other_columns + gev_other_columns + tev_other_columns + ['marked']
    
    catalog = pd.read_csv("markeddata/" + catalog_name + ".txt", sep='\t', index_col=0)
    xmm_catalog = pd.read_csv("data/{}.txt".format(xmm_catalog_name), sep='\t', index_col=0)
    catalog = catalog.join(xmm_catalog.loc[catalog.index, xmm_spectrum_columns])
    spectrum_columns = gev_spectrum_columns + s_spectrum_columns + tev_spectrum_columns + xmm_spectrum_columns
    spectrums = catalog[spectrum_columns + other_columns].copy()
    spectrums = spectrums.fillna(0)
    if log:
        only_spectra = spectrums[spectrum_columns].copy()
        only_spectra = np.log(only_spectra + 1)
#         only_spectra = np.log(only_spectra)
#         only_spectra[(only_spectra == -np.inf)] = 0
        spectrums[spectrum_columns] = only_spectra

    spectrums[spectrum_columns] = spectrums[spectrum_columns].div(spectrums[spectrum_columns].sum(axis=1), axis=0)
    spectrums = spectrums.fillna(0)
    display(spectrums)
    if ~log:
        add_name = "WITHOUT_LOG"
    spectrums.to_csv("markeddata/" + output_catalog_name + "_normalized" + add_name + ".txt", index=False, sep='\t')

* adding xmm spectrum columns to the gevtevsimbad, tevsimbad and gevsimbad spectrums

In [29]:
# calculate_spectra(catalog_name="gevtevsimbad_TeVspectrum", 
#                   xmm_catalog_name="gevtevsimbadxmm",
#                   output_catalog_name="gevtevsimbadxmm_spectrum",
#                   s_spectrum_columns=[], 
#                   gev_spectrum_columns=gev_spectrum_columns, 
#                   tev_spectrum_columns=tev_spectrum_columns,
#                   xmm_spectrum_columns=xmm_spectrum_columns,
#                   spectrum_columns=spectrum_columns,
#                   s_other_columns=[],
#                   gev_other_columns=gev_other_columns,
#                   tev_other_columns=tev_other_columns)
calculate_spectra(catalog_name="gevtevsimbad_TeVspectrum", 
                  xmm_catalog_name="gevtevsimbadxmm",
                  output_catalog_name="gevtevsimbadxmm_spectrum",
                  s_spectrum_columns=[], 
                  gev_spectrum_columns=gev_spectrum_columns, 
                  tev_spectrum_columns=tev_spectrum_columns,
                  xmm_spectrum_columns=xmm_spectrum_columns,
                  spectrum_columns=spectrum_columns,
                  s_other_columns=[],
                  gev_other_columns=gev_other_columns,
                  tev_other_columns=tev_other_columns)
calculate_spectra(catalog_name="tevsimbad_TeVspectrum",
                  xmm_catalog_name="tevsimbadxmm",
                  output_catalog_name="tevsimbadxmm_spectrum",
                  s_spectrum_columns =s_spectrum_columns, 
                  gev_spectrum_columns=[], 
                  tev_spectrum_columns=tev_spectrum_columns, 
                  xmm_spectrum_columns=xmm_spectrum_columns,
                  spectrum_columns=spectrum_columns,
                  s_other_columns=s_other_columns,
                  gev_other_columns=[],
                  tev_other_columns=tev_other_columns)
calculate_spectra(catalog_name="gevsimbad", 
                  xmm_catalog_name="gevsimbadxmm",
                  output_catalog_name="gevsimbadxmm_spectrum",
                  s_spectrum_columns=s_spectrum_columns, 
                  gev_spectrum_columns=gev_spectrum_columns, 
                  tev_spectrum_columns=[], 
                  xmm_spectrum_columns=xmm_spectrum_columns,
                  spectrum_columns=spectrum_columns,
                  s_other_columns=s_other_columns,
                  gev_other_columns=gev_other_columns,
                  tev_other_columns=[])

,gev_nuFnu10000_100000,gev_nuFnu1000_3000,gev_nuFnu100_300,gev_nuFnu3000_10000,gev_nuFnu300_1000,gev_nuFnu30_100,tev_0.3TeV,tev_1TeV,tev_3TeV,tev_10TeV,...,xmm_SC_EP_5_FLUX,gev_1FGL_Name,gev_CLASS1,gev_GLON,gev_GLAT,tev_fermi_names,tev_classes,tev_glat,tev_glon,marked
0,0.013311,0.337365,0.020479,0.140622,0.324914,0.0,0.151398,0.011056,0.000829,0.000025,...,0.000000,b' ',b' ',359.863953,-0.020071,b'',b'unid',-0.046162,359.944244,0
1,0.140280,0.140176,0.008468,0.152026,0.050583,0.0,0.421868,0.080696,0.005592,0.000291,...,0.000000,b' ',b'PWN ',26.795933,-0.198590,b'',b'unid',-0.197604,26.795113,0
2,0.013311,0.337365,0.020479,0.140622,0.324914,0.0,0.151398,0.011056,0.000829,0.000025,...,0.000000,b' ',b' ',359.863953,-0.020071,b'',b'unid',-0.046162,359.944244,0
3,0.034048,0.385752,0.003723,0.211482,0.245029,0.0,0.119149,0.000602,0.000201,0.000013,...,0.000000,b'1FGL J1746.4-2849c',b'pwn ',0.149201,-0.103157,b'',b'unid',-0.113206,0.140617,0
4,0.013311,0.337365,0.020479,0.140622,0.324914,0.0,0.151398,0.011056,0.000829,0.000025,...,0.000000,b' ',b' ',359.863953,-0.020071,b'',b'unid',-0.046162,359.944244,0
5,0.038689,0.438338,0.004231,0.240311,0.278431,0.0,0.000000,0.000000,0.000000,0.000000,...,0.000000,b'1FGL J1746.4-2849c',b'pwn ',0.149201,-0.103157,b'',b'unid',-0.147977,0.054990,0
6,0.034048,0.385752,0.003723,0.211482,0.245029,0.0,0.119149,0.000602,0.000201,0.000013,...,0.000000,b'1FGL J1746.4-2849c',b'pwn ',0.149201,-0.103157,b'',b'unid',-0.113206,0.140617,0
7,0.022409,0.362261,0.111730,0.253947,0.249653,0.0,0.000000,0.000000,0.000000,0.000000,...,0.000000,b'1FGL J2032.2+4127 ',b'PSR ',80.218597,1.018827,b'',b'unid',1.111117,80.299477,1
8,0.022409,0.362261,0.111730,0.253947,0.249653,0.0,0.000000,0.000000,0.000000,0.000000,...,0.000000,b'1FGL J2032.2+4127 ',b'PSR ',80.218597,1.018827,b'',b'unid',1.111117,80.299477,0
9,0.013692,0.204153,0.312633,0.093662,0.304338,0.0,0.070041,0.001341,0.000137,0.000004,...,0.000000,b'1FGL J0240.5+6113 ',b'HMB ',135.678848,1.086277,b'',b'bin',1.086135,135.675278,1


,gev_nuFnu10000_100000,gev_nuFnu1000_3000,gev_nuFnu100_300,gev_nuFnu3000_10000,gev_nuFnu300_1000,gev_nuFnu30_100,tev_0.3TeV,tev_1TeV,tev_3TeV,tev_10TeV,...,xmm_SC_EP_5_FLUX,gev_1FGL_Name,gev_CLASS1,gev_GLON,gev_GLAT,tev_fermi_names,tev_classes,tev_glat,tev_glon,marked
0,0.013311,0.337365,0.020479,0.140622,0.324914,0.0,0.151398,0.011056,0.000829,0.000025,...,0.000000,b' ',b' ',359.863953,-0.020071,b'',b'unid',-0.046162,359.944244,0
1,0.140280,0.140176,0.008468,0.152026,0.050583,0.0,0.421868,0.080696,0.005592,0.000291,...,0.000000,b' ',b'PWN ',26.795933,-0.198590,b'',b'unid',-0.197604,26.795113,0
2,0.013311,0.337365,0.020479,0.140622,0.324914,0.0,0.151398,0.011056,0.000829,0.000025,...,0.000000,b' ',b' ',359.863953,-0.020071,b'',b'unid',-0.046162,359.944244,0
3,0.034048,0.385752,0.003723,0.211482,0.245029,0.0,0.119149,0.000602,0.000201,0.000013,...,0.000000,b'1FGL J1746.4-2849c',b'pwn ',0.149201,-0.103157,b'',b'unid',-0.113206,0.140617,0
4,0.013311,0.337365,0.020479,0.140622,0.324914,0.0,0.151398,0.011056,0.000829,0.000025,...,0.000000,b' ',b' ',359.863953,-0.020071,b'',b'unid',-0.046162,359.944244,0
5,0.038689,0.438338,0.004231,0.240311,0.278431,0.0,0.000000,0.000000,0.000000,0.000000,...,0.000000,b'1FGL J1746.4-2849c',b'pwn ',0.149201,-0.103157,b'',b'unid',-0.147977,0.054990,0
6,0.034048,0.385752,0.003723,0.211482,0.245029,0.0,0.119149,0.000602,0.000201,0.000013,...,0.000000,b'1FGL J1746.4-2849c',b'pwn ',0.149201,-0.103157,b'',b'unid',-0.113206,0.140617,0
7,0.022409,0.362261,0.111730,0.253947,0.249653,0.0,0.000000,0.000000,0.000000,0.000000,...,0.000000,b'1FGL J2032.2+4127 ',b'PSR ',80.218597,1.018827,b'',b'unid',1.111117,80.299477,1
8,0.022409,0.362261,0.111730,0.253947,0.249653,0.0,0.000000,0.000000,0.000000,0.000000,...,0.000000,b'1FGL J2032.2+4127 ',b'PSR ',80.218597,1.018827,b'',b'unid',1.111117,80.299477,0
9,0.013692,0.204153,0.312633,0.093662,0.304338,0.0,0.070041,0.001341,0.000137,0.000004,...,0.000000,b'1FGL J0240.5+6113 ',b'HMB ',135.678848,1.086277,b'',b'bin',1.086135,135.675278,1


,s_FLUX_U,s_FLUX_B,s_FLUX_V,s_FLUX_R,s_FLUX_I,s_FLUX_G,s_FLUX_J,s_FLUX_H,s_FLUX_K,s_FLUX_u,...,xmm_SC_EP_4_FLUX,xmm_SC_EP_5_FLUX,s_MAIN_ID,s_class,s_nan_spectra,tev_fermi_names,tev_classes,tev_glat,tev_glon,marked
0,0.000000,0.000000,0.000000,0.205052,0.000000,0.203444,0.198642,0.197336,0.195525,0.0,...,0.000000e+00,0.000000e+00,b'UCAC4 105-013663',Be,False,b'',b'other',-31.911823,279.623993,0
1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.352144,0.333805,0.314051,0.0,...,0.000000e+00,0.000000e+00,b'MSX6C G331.5487-00.5364',Be,False,b'',b'unid',-0.573211,331.512299,0
2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.354718,0.334432,0.310850,0.0,...,0.000000e+00,0.000000e+00,b'SSTGC 519103',Be,False,b'',b'unid',-0.046162,359.944244,0
3,0.000000,0.202976,0.201722,0.000000,0.000000,0.000000,0.199032,0.198375,0.197894,0.0,...,0.000000e+00,0.000000e+00,b'RMC 141',B,False,b'',b'pwn',-31.745857,279.558716,0
4,0.000000,0.124692,0.124509,0.125217,0.124828,0.124510,0.125080,0.125545,0.125619,0.0,...,0.000000e+00,0.000000e+00,b'[AAA97b] J053648.7-691700',B,False,b'',b'other',-31.911823,279.623993,0
5,0.000000,0.125182,0.124978,0.124953,0.124244,0.124539,0.125360,0.125309,0.125436,0.0,...,0.000000e+00,0.000000e+00,b'OGLE LMC-ECL-20994',B,False,b'',b'pwn',-31.745857,279.558716,0
6,0.329412,0.335021,0.335567,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.000000e+00,0.000000e+00,b'[ST92] 2-62B',B,False,b'',b'other',-31.911823,279.623993,0
7,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.339721,0.332055,0.328224,0.0,...,0.000000e+00,0.000000e+00,b'[KKM2013] 7',B,False,b'',"b'pwn,snr'",0.260453,54.094742,0
8,0.334505,0.337459,0.328036,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.000000e+00,0.000000e+00,b'[ST92] 2-62',B,False,b'',b'other',-31.911823,279.623993,0
9,0.114003,0.115548,0.113658,0.112294,0.106379,0.113546,0.109130,0.108093,0.107349,0.0,...,0.000000e+00,0.000000e+00,b'HTR 13',B,False,b'',b'pwn',-31.745857,279.558716,0


,gev_nuFnu10000_100000,gev_nuFnu1000_3000,gev_nuFnu100_300,gev_nuFnu3000_10000,gev_nuFnu300_1000,gev_nuFnu30_100,s_FLUX_U,s_FLUX_B,s_FLUX_V,s_FLUX_R,...,xmm_SC_EP_4_FLUX,xmm_SC_EP_5_FLUX,s_MAIN_ID,s_class,s_nan_spectra,gev_1FGL_Name,gev_CLASS1,gev_GLON,gev_GLAT,marked
0,0.016566,0.318117,1.167134e-01,0.194983,0.353621,0.0,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,b'[KRL2007b] 194',Be,True,b'1FGL J1636.4-4737c',b'snr ',337.030243,-0.156472,0
1,0.000004,0.000080,1.745034e-05,0.000040,0.000080,0.0,0.000087,0.000036,0.000081,0.000102,...,0.0,0.0,b'HD 328990',Be,False,b'1FGL J1651.5-4602c',b' ',339.797699,-0.952196,0
2,0.014352,0.087268,9.648025e-02,0.058873,0.082640,0.0,0.000042,0.000023,0.000179,0.000818,...,0.0,0.0,b'UCAC2 5051621',Be,False,b' ',b'spp ',291.034882,-0.087617,0
3,0.054147,0.123705,4.429593e-01,0.121311,0.188794,0.0,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,b'SSTGC 519103',Be,False,b'1FGL J1745.6-2900c',b'spp ',359.955597,-0.039297,0
4,0.046426,0.232215,2.710143e-01,0.145265,0.304360,0.0,0.000000,0.000031,0.000023,0.000000,...,0.0,0.0,b'LIN 341',Be,False,b'1FGL J0101.3-7257 ',b'GAL ',302.145142,-44.416740,0
5,0.001524,0.006400,1.623112e-02,0.004016,0.008842,0.0,0.150875,0.034564,0.052314,0.000000,...,0.0,0.0,b'LS VI -04 15',B,False,b' ',b' ',217.957901,-0.395969,0
6,0.000157,0.002620,8.954836e-03,0.001090,0.003881,0.0,0.000000,0.006988,0.015287,0.000000,...,0.0,0.0,b'LS I +56 58',B,False,b' ',b' ',136.883270,-3.233810,0
7,0.000000,0.131294,4.130467e-01,0.074252,0.370510,0.0,0.000000,0.000225,0.000451,0.000301,...,0.0,0.0,b'PG 1101+529',B,False,b' ',b' ',153.787582,57.501030,0
8,0.000000,0.240205,3.359972e-01,0.083134,0.324734,0.0,0.000000,0.000000,0.015930,0.000000,...,0.0,0.0,b'GALEX J175340.5-500741',B,False,b'1FGL J1754.0-5002 ',b'bcu ',342.278839,-11.987718,0
9,0.019413,0.118045,1.305058e-01,0.079636,0.111785,0.0,0.002012,0.002059,0.007650,0.014575,...,0.0,0.0,b'HD 306182',B,False,b' ',b'spp ',291.034882,-0.087617,0


* contain objects, which have spectrum information from catalogs (gev, tev, simbad) and are in their intersection, but were not found in xmm catalog

In [30]:
# cat_name = 'gevtevsimbadxmm_001'
cat_name = 'gevtevsimbadxmm_005'
# cat_name = 'gevtevsimbadxmm_012'

normalize_spectra(catalog_name=cat_name, 
                   s_spectrum_columns=xmm_spectrum_columns, 
                   gev_spectrum_columns=gev_spectrum_columns, 
                   tev_spectrum_columns=tev_spectrum_columns, 
                   s_other_columns=xmm_other_columns + ['s_MAIN_ID', 's_simbad', 's_class'],
                   gev_other_columns=gev_other_columns,
                   tev_other_columns=tev_other_columns,
                   log = False)

,xmm_SC_EP_1_FLUX,xmm_SC_EP_2_FLUX,xmm_SC_EP_3_FLUX,xmm_SC_EP_4_FLUX,xmm_SC_EP_5_FLUX,gev_nuFnu10000_100000,gev_nuFnu1000_3000,gev_nuFnu100_300,gev_nuFnu3000_10000,gev_nuFnu300_1000,...,s_class,gev_1FGL_Name,gev_CLASS1,gev_GLON,gev_GLAT,tev_fermi_names,tev_classes,tev_glat,tev_glon,marked
0,1.453264e-04,1.162377e-03,0.008684,0.005385,3.485449e-05,0.068054,0.072044,0.003354,0.037734,0.143372,...,[],b' ',b'snr ',120.125992,1.337755,b'',b'snr',1.412930,120.092361,0
1,1.246025e-05,1.615568e-03,0.010141,0.006539,0.000000e+00,0.067854,0.071832,0.003344,0.037623,0.142950,...,[],b' ',b'snr ',120.125992,1.337755,b'',b'snr',1.412930,120.092361,0
2,4.486808e-05,2.262883e-03,0.001160,0.000055,0.000000e+00,0.068876,0.072914,0.003395,0.038190,0.145103,...,[],b' ',b'snr ',120.125992,1.337755,b'',b'snr',1.412930,120.092361,0
3,8.961457e-05,3.330991e-03,0.005777,0.002133,5.083657e-06,0.068336,0.072342,0.003368,0.037890,0.143965,...,[],b' ',b'snr ',120.125992,1.337755,b'',b'snr',1.412930,120.092361,0
4,8.252238e-04,1.496075e-02,0.043244,0.026935,1.251375e-04,0.063169,0.066872,0.003113,0.035025,0.133080,...,[],b' ',b'snr ',120.125992,1.337755,b'',b'snr',1.412930,120.092361,0
5,1.349531e-04,2.965410e-03,0.006451,0.001340,4.606617e-04,0.068335,0.072341,0.003368,0.037890,0.143963,...,[],b' ',b'snr ',120.125992,1.337755,b'',b'snr',1.412930,120.092361,0
6,4.005552e-05,1.401393e-03,0.005904,0.001996,5.708213e-05,0.068470,0.072484,0.003375,0.037964,0.144247,...,[],b' ',b'snr ',120.125992,1.337755,b'',b'snr',1.412930,120.092361,0
7,1.827455e-03,2.988558e-02,0.058672,0.026196,7.439414e-05,0.061056,0.064636,0.003009,0.033854,0.128629,...,[],b' ',b'snr ',120.125992,1.337755,b'',b'snr',1.412930,120.092361,0
8,9.313951e-05,2.526046e-03,0.005589,0.003593,0.000000e+00,0.068304,0.072308,0.003366,0.037872,0.143897,...,[],b' ',b'snr ',120.125992,1.337755,b'',b'snr',1.412930,120.092361,0
9,1.484815e-04,2.599756e-03,0.004453,0.002210,0.000000e+00,0.068469,0.072483,0.003375,0.037964,0.144245,...,[],b' ',b'snr ',120.125992,1.337755,b'',b'snr',1.412930,120.092361,0


In [28]:
# calculate_spectra(catalog_name="gevtev_TeVspectrum", 
#                   s_spectrum_columns=[], 
#                   gev_spectrum_columns=gev_spectrum_columns, 
#                   tev_spectrum_columns=tev_spectrum_columns, 
#                   s_other_columns=[],
#                   gev_other_columns=gev_other_columns,
#                   tev_other_columns=tev_other_columns)
# calculate_spectra(catalog_name="tevsimbad_TeVspectrum", 
#                   s_spectrum_columns =s_spectrum_columns, 
#                   gev_spectrum_columns=[], 
#                   tev_spectrum_columns=tev_spectrum_columns, 
#                   s_other_columns=s_other_columns,
#                   gev_other_columns=[],
#                   tev_other_columns=tev_other_columns)
# calculate_spectra(catalog_name="gevsimbad", 
#                   s_spectrum_columns=s_spectrum_columns, 
#                   gev_spectrum_columns=gev_spectrum_columns, 
#                   tev_spectrum_columns=[], 
#                   s_other_columns=s_other_columns,
#                   gev_other_columns=gev_other_columns,
#                   tev_other_columns=[])